In [2]:
import numpy as np
from typeguard import typechecked
from decimal import Decimal
from IPython.display import display, Latex
import numbers
from uncertainties import ufloat
import uncertainties
from pint import UnitRegistry, DimensionalityError
ureg = UnitRegistry()
ureg.default_format = "~P"
Q_=ureg.Quantity
from interval import interval, inf, imath

esc = lambda s: s.replace('"\"','"\\"')


In [3]:
class Limits:
    def __init__(self, limits):
        lower, upper = limits
        try:
            assert lower.dimensionality == upper.dimensionality
        except DimensionalityError as exc:
            raise exc
        try:
            assert isinstance(lower.m, numbers.Number)
            assert isinstance(upper.m, numbers.Number)
        except AssertionError as e:
            print('lower and upper limits must be integer of floats')
            raise e
        lower.ito_base_units()
        upper.ito_base_units()
        self.value = interval([lower.m, upper.m])
        self.units = lower.units
    
    def __mul__(self,other):
        if isinstance(other, numbers.Number):
            return self.value*other
        elif isinstance(other, Limits):
            return self.value*other.value

    def __rmul__(self,other):
        return self.__mul__(other)
    
    def __truediv__(self,other):
        if isinstance(other, numbers.Number):
            return self.value/other
        elif isinstance(other, Limits):
            return self.value/other.value
        
    def __str__(self):
        h = [(x[0].inf,x[0].sup) for x in self.value.components]
        h = "&".join([f"{x}\, — {y}" for x,y in h])
        return h + '\\, '+ self.units.__str__()
    
    def __repr__(self):
        s = f"$${self.__str__()}$$"
        display(Latex(s))
        return ""    

In [23]:
class BasicQuantity:
    __significant_digits__ = 3
    units = '1'
    class_units = Q_(f"{units}")
    def __init__(self, name="", **kwargs):
        c1 = set(kwargs.keys()) == {'magnitude','units'}
        c2 = set(kwargs.keys()) == {'quantity'}
        c3 = set(kwargs.keys()) == {'limits'}
        self.name = name

        assert c1 ^ c2 ^ c3
        if c1:
            magnitude = kwargs['magnitude']
            units = kwargs['units']
            if isinstance(magnitude,numbers.Number):
                self.magnitude = ufloat(magnitude,0)
            else:
                self.magnitude = magnitude
                self.units = units
            self.value = Q_(self.magnitude, self.units)
        if c2:
            quantity = kwargs['quantity']
            self.value = quantity          
        if c3:
            limits = kwargs['limits']
            self.value = limits 
        
        if c1^c2:
            try:
                assert self.class_units.dimensionality == self.value.dimensionality
            except AssertionError:
                raise DimensionalityError(self.class_units.units, self.value.units)

    def __eq__(self, other):
        try:
            x = self.value.to_base_units()
            y = other.value.to_base_units()
            assert x.units == y.units
            assert __zeta__(x.m, y.m)
            return True
        except:
            return False

    def __add__(self,other):
        try:
            return self.value + other.value
        except DimensionalityError as exc:
            raise exc

    def __sub__(self, other):
        try:
            return self.value - other.value
        except DimensionalityError as exc:
            raise exc
            
    def __mul__(self,other):
        if isinstance(other, numbers.Number):
            return self.value*other
        elif isinstance(other, BasicQuantity):
            return self.value*other.value

    def __rmul__(self,other):
        return self.__mul__(other)
    
    def __pow__(self,n):
        return self.value**n
        
    def __truediv__(self,other):
        if isinstance(other, numbers.Number):
            return self.value/other
        elif isinstance(other, BasicQuantity):
            return self.value/other.value
        
    def __str__(self):
        if isinstance(self.value, Limits):
            print('>>> Limits')
            h = [(x[0].inf,x[0].sup) for x in self.value.value.components]
            h = "&".join([f"{x}\, — {y}" for x,y in h])
            h =  h + '\\, '
            u = self.units
        elif isinstance(self.value, ureg.Quantity):
            print('>>> quantity')
            if isinstance(self.value.m, np.ndarray):
                print('>>> array')
                h = list(self.value.m).__str__()
                h = "["+", ".join([f"{x:.{self.__significant_digits__}u}" for x in self.value.m])+"]"
            else:
                print('>>> scalar', type(self.value.m))
                if isinstance(self.value.m, numbers.Number):
                    h = f"{self.value.m:4.{self.__significant_digits__}g}"
                else:
                    if self.value.m.std_dev == 0:
                        h = f"{self.value.m.nominal_value:4.{self.__significant_digits__}g}"
                    else:
                        h = f"{self.value.m:4.{self.__significant_digits__}g}"
            u = self.value.units
        s = f"{self.name} = {h} \\, \\rm{{[{u}]}}, \\, \\rm{{({self.__class__.__name__})}}"
        s = s.replace('+/-',r'\pm')
        return s
    
    def __repr__(self):
        s = f"$${self.__str__()}$$"
        display(Latex(s))
        return ""
    

def quantity_maker(klass, units, expression=lambda x:Q_('0')):
    return type(klass,(BasicQuantity,),{"class_units":Q_(units),'units':units ,'expression':expression})    

In [24]:
Mass = quantity_maker('Mass','kg')
m = Mass(name='m', limits=Limits([Q_('6 kg'), Q_('7 kg')]))
print(m)
display(m)

>>> Limits
m = 6.0\, — 7.0\,  \, \rm{[kg]}, \, \rm{(Mass)}
>>> Limits


<IPython.core.display.Latex object>

In [ ]:
# custom physical quantities    
Mass = quantity_maker('Mass','kg')
Time = quantity_maker('Mass','sec')
Length = quantity_maker('Length','m')
Area = quantity_maker('Area','m^2')
Volume = quantity_maker('Volume','m^3')

Porosity = quantity_maker('Porosity','m^3/m^3')
print(Porosity.units)
EffPorosity = quantity_maker('EffPorosity','m^3/m^3')
print(EffPorosity.units)

In [ ]:
Velocity = quantity_maker('Velocity','m/s')
@typechecked
def speed(name:str, l:Length, dt:Time) -> Velocity:
    name = f'{name} = \\frac{{{l.name}}}{{{dt.name}}}'
    q = l/dt
    return Velocity(name = name, quantity = q)

Density = quantity_maker('Density','kg/m^3')
@typechecked
def density(name:str, m:Mass, v:Volume) -> Density:
    name = f'{name} = \\frac{{{m.name}}}{{{v.name}}}'
    return Density(name = name, quantity = (m/v))

m = Mass(name='m', magnitude=ufloat(6, 0.1), units='kg')
display(m)

v = Volume(name='v', quantity=Q_(ufloat(3, 0.2),'m^3'))
display(v)

rho_1 = density(r'\rho',m,v)
display(rho_1)

m2 = Mass(name=r'm_2', magnitude=ufloat(6, 0.1), units='kg')
display(m2)

rho_2 = density(r'\rho_2',m2,v)
display(rho_2)

In [ ]:
m = Mass(name='m', magnitude=6, units='kg')
display(m)

In [ ]:
m0 = Mass(name='m_0', limits=Limits(['2.8 kg','6.07 kg']))
m0 = Mass(name='m_0', limits=Limits([Q_('2.8 kg'),Q_('6.07 kg')]))
print(m0)
v0 = Volume(name='v_0', limits=Limits([Q_('3.21 m^3'),Q_('3.6 m^3')]))
print(v0*m0)
rho_0 = density(r'\zeta',m0,v0)
display(rho_0)
print(rho_0)

In [35]:
r = Mass(r'\overline{m}',quantity=Q_(np.array([ufloat(3,0.1),ufloat(4,0.2)]),'kg'))
display(r)

>>> quantity
>>> array


<IPython.core.display.Latex object>

In [ ]:
m_vec = Mass(r'\bar{m}',quantity=Q_(np.array([ufloat(3,0.1),ufloat(4,0.2)]),'kg'))
v_vec = Volume(r'\bar{v}',quantity=Q_(np.array([ufloat(3,0.1),ufloat(4,0.2)]),'m^3'))
r_vec = density(r'\bar{\rho}',m_vec,v_vec)
display(r_vec)

In [ ]:
def mag_in(x:ureg.Quantity, u:str)->float:
    return float(x.value.to(u).m)

Permittivity = quantity_maker('Permittivity','1')
@typechecked
def dryrock_permittivity(name:str, rho:Density) -> Permittivity:
    # formula 1.91**rho
    # rho must be in gr/cm^3
    #     r = rho.to('g/cm^3').m
    r = mag_in(rho,'g/cm^3')
    name = f'\kappa = 1.91^{rho.name}'
    return Permittivity(name=name,magnitude=1.91**r,units='')
rho = Density(name=r'{\rho_0}', quantity=Q_('2.32 g/cm^3'))
mag_in(rho,'g/cm^3')

k_quartz = dryrock_permittivity('\kappa' , rho)
k_quartz_measured = 4.30 # https://academic.oup.com/gji/article/163/3/1195/2127261
display(k_quartz)

In [ ]:
Permittivity = quantity_maker('Permittivity','1')
@typechecked
def dryrock_permittivity(name:str, rho:Density) -> Permittivity:
    name = f'\kappa = exp(\\frac{{{rho.name}}}{{1.5455 [g/cm^3]}})'
    rho_ref = Density('',quantity = Q_('1.5455 g/cm^3'))
    p = np.exp(rho / rho_ref)
    return Permittivity(name=name,magnitude=p,units='')
rho = Density(name=r'{\rho_0}', quantity=Q_('2.32 g/cm^3'))

k_quartz = dryrock_permittivity('\kappa' , rho)
k_quartz_measured = 4.30 # https://academic.oup.com/gji/article/163/3/1195/2127261
display(k_quartz)

In [ ]:
BulkModulus = quantity_maker('BulkModulus','Pa')
ShearModulus = quantity_maker('ShearModulus','Pa')

from dataclasses import dataclass
@dataclass
class Mineral:
    density: Density
    bulk_modulus: BulkModulus
    shear_modulus: ShearModulus

@dataclass
class Composition:
    composition: list(float)
        
class MineralComposition(Composition):
    composition: list(Mineral)
    
@dataclass
class Fluid:
    density:
    viscosity:
        
@dataclass
class Rock:
    mineral_composition: MineralComposition
    fluid_composition: FluidComposition
    compressional_velocity: 
        

@typechecked